## This is code for calculating weingarten function

#Full Process
## 1. Use sympy to symbolically express the purity of boson sampling.
## 2. Convert the results of sympy to mathematica language.(Bucause I use mathematica package IntU for calculating wiengarten fuction, we need this process. If you find other good package on python, you don't need this process)
## 3. By using Wolfram Client Library for python, We calculate weingarten function

In [1]:
import sympy
from sympy import MatrixSymbol, Matrix, Transpose, shape, simplify
from sympy.physics.quantum.dagger import Dagger
from sympy.matrices import zeros, ones
from sympy import latex

In [2]:
from wolframclient.evaluation import WolframLanguageSession
from wolframclient.language import wl, wlexpr

In [18]:
from fractions import Fraction
from sympy.printing import pycode
import re
import sympy as sp

# ----------------
# 1. Calculating purity of the weingartne function

In [4]:
import numpy as np

# set the random seed
np.random.seed(42)

# import Strawberry Fields
import strawberryfields as sf
from strawberryfields.ops import *
from strawberryfields import ops

import itertools
from math import comb, sqrt, factorial

from numpy import ndarray

https://strawberryfields.readthedocs.io/en/stable/code/api/strawberryfields.utils.random_interferometer.html

In [5]:
def fock_generator(q, i):
    ops.Fock(1) | q[i]

In [6]:
def vac_generator(q, N, i):
    ops.Vac | q[N + i]

In [ ]:
# Here we just make a circuit for boson sampling. 
def circuit_generator(N, M): #N is the number of single photons and M is the mode number. 
    U = sf.utils.random_interferometer(M) 
    
    U_s = Matrix(MatrixSymbol('U', M, M))

    # initialize a 4 mode program
    boson_sampling = sf.Program(M) #This is just initialization for making quantum circuit for photon. Not for just boson sampling. It is just initialization. 

    with boson_sampling.context as q:
        # prepare the input fock states
        for i in range(N):
            fock_generator(q, i)
        for i in range(M-N):
            vac_generator(q, N, i)

        ops.Interferometer(U) | q
    return U, U_s, boson_sampling


In [8]:
# Compute the coefficients for purity
def compute_coefficients(s, M):
    # Padding s with zeros if M > len(s)
    if M > len(s):
        s += [0] * (M - len(s))
    #print(s)
    
    S = sum(s)
    v = [range(i+1) for i in s]
    #print(v)
    combinations = list(itertools.product(*v))
    
    A = []
    xi = []
    x = []
    xi_no_z = []
    x_no_z = []
    
    for combo in combinations:
        h = [(s[i]/2) - combo[i] for i in range(M)]
        h_1 = [(s[i]/2) - combo[i] for i in range(S)]
        #print(h)
        
        part1 = (-1)**sum(combo)
        part2 = np.prod([comb(s[i], combo[i]) for i in range(M)])
        part3 = (sum([abs(h_i)**2 for h_i in h]) ** (S/2)) / sqrt(factorial(S))
        
        A_k = part1 * part2 * part3
        xi_k = [h_i / (sum([abs(h_j)**2 for h_j in h])** (S/2)) for h_i in h]
        xi_k_1 = [h_i / (sum([abs(h_j)**2 for h_j in h_1])** (S/2)) for h_i in h_1]
        x_k = [h_i *2 for h_i in h]
        x_k_1 = [h_i *2 for h_i in h_1]

        A.append(A_k)
        xi.append(xi_k)
        x.append(x_k)
        xi_no_z.append(xi_k_1)
        x_no_z.append(x_k_1)
    
    return np.array(A), np.array(xi), np.array(x), np.array(xi_no_z), np.array(x_no_z)

## Generate $\Lambda_L$ and $\Lambda_R$

In [9]:
def Lambda_generator(U, U_s,S,ML):
    u_l = U[0:S, 0:ML]
    u_l_s = U_s[0:S, 0:ML]
    u_l_d = u_l.conj().T
    u_l_d_s = Dagger(u_l_s)
    lambda_L = np.matmul(u_l, u_l_d)
    lambda_L_s = u_l_s@u_l_d_s
    u_r = U[0:S, ML:]
    u_r_s = U_s[0:S, ML:]
    u_r_d = u_r.conj().T
    u_r_d_s = Dagger(u_r_s)
    lambda_R = np.matmul(u_r, u_r_d)
    lambda_R_s = u_r_s@u_r_d_s
    return lambda_L, lambda_R, lambda_L_s, lambda_R_s

## Calculate $Tr(\rho^2)$

In [10]:
def tr_rho_L2(x_no_z, S, Lambda_L, Lambda_R, Lambda_L_s, Lambda_R_s):
    save = []
    save_coeff = []
    result = 0
    result_s = zeros(1)
    factor = (1/2)**(4*S)
    #print(factor)
    #print(x_no_z, S, Lambda_L, Lambda_R, Lambda_L_s, Lambda_R_s) #이걸 검사해본 결과, Lambda_L_s

    # Calculate each term in the summation
    for n in range(S+1):
        term1 = 1 / (factorial(S-n)*factorial(n))**2
        save_coeff.append(factor * term1)
        total_sum = 0
        total_sum_s = zeros(1)

        # Iterate through all the indices k, l, k', l'
        for k, l, k_prime, l_prime in itertools.product(range(2**S), repeat=4):
            prod = 1

            for i in range(S):
                x_k = x_no_z[k][i]
                x_l = x_no_z[l][i]
                x_k_prime = x_no_z[k_prime][i]
                x_l_prime = x_no_z[l_prime][i]
                
                prod *= x_k * x_l * x_k_prime * x_l_prime

            term2 = prod * np.power(np.dot(np.dot(x_no_z[k_prime], Lambda_L), x_no_z[l][:,None]) * np.dot(np.dot(x_no_z[k], Lambda_L), x_no_z[l_prime][:, None]), S-n)
            term2_s = prod * (Transpose(Matrix(x_no_z[k_prime])) @ Lambda_L_s @ Matrix(x_no_z[l]) @ Transpose(Matrix(x_no_z[k])) @ Lambda_L_s @ Matrix(x_no_z[l_prime]))**(S-n)
            term3 = np.power(np.dot(np.dot(x_no_z[k], Lambda_R), x_no_z[l][:,None]) * np.dot(np.dot(x_no_z[k_prime], Lambda_R), x_no_z[l_prime][:,None]), n)
            term3_s = (Transpose(Matrix(x_no_z[k]))@Lambda_R_s@Matrix(x_no_z[l])@Transpose(Matrix(x_no_z[k_prime]))@Lambda_R_s@Matrix(x_no_z[l_prime]))**n
            total_sum += term2 * term3
            total_sum_s += term2_s*term3_s
            # save.append(term3)

        result += term1 * total_sum
        result_s += term1* total_sum_s
    
    return factor * result, factor * result_s

## result_s is a form of multiplication, result_s_exp is a result after we expand all equation of the result_s.

In [11]:
def purity_calculator(s, M, ML):
    S = sum(s)
    U, U_s, boson_sampling = circuit_generator(S, M)
    A, xi, x, xi_no_z, x_no_z = compute_coefficients(s, M)
    Lambda_L, Lambda_R, Lambda_L_s, Lambda_R_s = Lambda_generator(U, U_s,S,ML)
    result, result_s = tr_rho_L2(x_no_z, S, Lambda_L, Lambda_R, Lambda_L_s, Lambda_R_s)
    return U, result, U_s, sympy.expand(result_s), result_s

# Like below, you can get a purity of the boson sampling

In [12]:
s = [1,1] # Example
M = 4
ML = int(M/2)
U, result, U_s, result_s_exp, result_s = purity_calculator(s, M, ML)

# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ

## The code below is a code that changes the sympy result to mathematica language.

In [15]:
def sympy_to_mathematica_complete_with_integration(sympy_expr):
    # Pattern to match \overline{U_{i, j}}^{k} format
    pattern_exponent_conjugate = r"\\overline{U_{(\d+), (\d+)}}\^{(\d+)}"
    # Pattern to match \overline{U_{i, j}} format
    pattern_simple_conjugate = r"\\overline{U_{(\d+), (\d+)}}"
    # Pattern to match U_{i, j}^{k} format
    pattern_exponent = r"U_{(\d+), (\d+)}\^{(\d+)}"
    # Pattern to match U_{i, j} format
    pattern_simple = r"U_{(\d+), (\d+)}"

    # Function to replace matched conjugate patterns with exponent
    def repl_exponent_conjugate(match):
        i, j, k = match.groups()
        return f"Conjugate[Subscript[u, {i}, {j}]]^{k}"

    # Function to replace matched simple conjugate patterns
    def repl_simple_conjugate(match):
        i, j = match.groups()
        return f"Conjugate[Subscript[u, {i}, {j}]]"

    # Function to replace matched patterns with exponent (non-conjugate)
    def repl_exponent(match):
        i, j, k = match.groups()
        return f"Subscript[u, {i}, {j}]^{k}"

    # Function to replace matched simple patterns (non-conjugate)
    def repl_simple(match):
        i, j = match.groups()
        return f"Subscript[u, {i}, {j}]"

    # Apply replacements in a sequence to avoid duplications
    transformed_expr = re.sub(pattern_exponent_conjugate, repl_exponent_conjugate, sympy_expr)
    transformed_expr = re.sub(pattern_simple_conjugate, repl_simple_conjugate, transformed_expr)
    transformed_expr = re.sub(pattern_exponent, repl_exponent, transformed_expr)
    transformed_expr = re.sub(pattern_simple, repl_simple, transformed_expr)

    # Adding integration syntax
    return f"IntegrateUnitaryHaar[{transformed_expr}, {{u, d}}]"

# Example usage
example_sympy_expr = '\\overline{U_{0, 0}} \\overline{U_{0, 1}} \\overline{U_{1, 0}}^{2} U_{0, 0}^{2} U_{1, 0} U_{1, 1}'
mathematica_expr_with_integration = sympy_to_mathematica_complete_with_integration(example_sympy_expr)
mathematica_expr_with_integration


'IntegrateUnitaryHaar[Conjugate[Subscript[u, 0, 0]] Conjugate[Subscript[u, 0, 1]] Conjugate[Subscript[u, 1, 0]]^2 Subscript[u, 0, 0]^2 Subscript[u, 1, 0] Subscript[u, 1, 1], {u, d}]'

## ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ

## It works only if you own Mathematica, and will not work if Mathematica is running separately.

## ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ

## In the code below, we cut off the sympy results whenever + and - occur and store them in a list. Each element in the stored list is converted into a dictionary with coefficient as the key and element of unitary as the key. Convert the key value to mathematica langauge and calculate the weingarten function using the mathematica IntU package. Calculate the expected value of purity by multiplying the calculated result by the coefficient and then adding them all together.

In [19]:
def auto_mathematica(s, M, ML, kernel_path, path_to_intu):
    result_U = []
    result_wein = []
    U, result, U_s, result_s_exp, result_s = purity_calculator(s, M, ML)
    ##### 아래 부분은 weingarten function을 다 잘라서 listfh 저장하는거. ex) 3A + 4B -> [3A, 4B]
    trace_expr = result_s_exp[0]
    # Check if the expression is an instance of sympy.core.add.Add
    if isinstance(trace_expr, sp.core.add.Add):
        # Split the expression into separate terms and store them in a list
        terms = list(trace_expr.as_ordered_terms())
    else:
        # If the expression is not a sum, store it as the only element in the list
        terms = [trace_expr]
    
    #### 아래 부분은 나눠진 각각을 coeffocient와 unitary 부분으로 나눈거 ex) ->  
    coeff_ml_M_2 = []

    for i in range(len(terms)):
        coeff_ml_M_2.append(terms[i].as_coefficients_dict())
    #### 아래 부분은 unitary 부분을 mathematica에서 사용 가능한 format으로 바꾼 부분
    with WolframLanguageSession(kernel_path) as session:
    # Load the IntU package
        session.evaluate(wlexpr(f'<< "{path_to_intu}"'))

        # Set the value of d
        session.evaluate(wlexpr(f'd = {M};'))

        # Define your Mathematica command
        #math_command = 'IntegrateUnitaryHaar[Conjugate[Subscript[u, 0, 0]]^2 Conjugate[Subscript[u, 1, 0]]^2 Subscript[u, 0, 0]^2 Subscript[u, 1, 0]^2, {u, d}]'
        for j in range(len(terms)):
            math_command = sympy_to_mathematica_complete_with_integration(latex(list(coeff_ml_M_2[j].keys())[0]))

            result = session.evaluate(wlexpr(math_command))

            if isinstance(result, type(wl.Rational(1, 210))):
                # Extracting data from the WLFunction object
                numerator = result.args[0]
                denominator = result.args[1]
                fraction_result = Fraction(numerator, denominator)
            else:
                # Directly create a Fraction from the output
                fraction_result = Fraction(result)

            result_U.append(fraction_result)
            result_wein.append(fraction_result* list(coeff_ml_M_2[j].values())[0])
    purity = 0
    for k in range(len(result_wein)):
        purity += result_wein[k]
    return purity, result_U, result_wein, coeff_ml_M_2

In [20]:
s = [1,1] # Example
M = 4
ML = 1
kernel_path = '/Applications/Mathematica.app/Contents/MacOS/WolframKernel'
path_to_intu = "/Users/kimjaehee/IntU_copy/IntU.m"
purity, result_U, result_wein, coeff_ml_M_2 = auto_mathematica(s, M, ML, kernel_path, path_to_intu)
print(purity)

0.514285714285715


In [123]:
s = [1,1] # Example
M = 5
ML = 1
kernel_path = '/Applications/Mathematica.app/Contents/MacOS/WolframKernel'
path_to_intu = "/Users/kimjaehee/IntU_copy/IntU.m"
purity, result_U, result_wein, coeff_ml_M_2 = auto_mathematica(s, M, ML, kernel_path, path_to_intu)
print(purity)

0.561904761904762


## ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ